## Import modules

In [50]:
import pandas as pd
import os
import numpy as np
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
module_path = os.path.abspath(os.path.join('..'))

## Load and preprocess

In [51]:
all_data = pd.ExcelFile(module_path + "/data/harel/tableS1.xlsx")

clinical_response_df = all_data.parse('S1A', header = 1) #load in clinical and response data

response_df = clinical_response_df.loc[:,['Sample ID','Response', 'PFS time (months)', 'OS time (months)']] #response data

clinical_df = clinical_response_df.drop(columns = ['Response', 'PFS time (months)', 'OS time (months)']) #clinical data

protein_df = all_data.parse('S1B', header = 1) #load in protein data
protein_df = protein_df.drop(columns = ['T: Protein IDs', 'T: Majority protein IDs', 'T: Protein names', 'T: ENSG'])
protein_df = protein_df.dropna(subset=['T: Gene names']).reset_index(drop = True) #drop unlabeled proteins

drop_idx_noname = [i for i,j in enumerate(protein_df['T: Gene names']) if ';' in j] #drop rows with multiple protein names
protein_df = protein_df.drop(drop_idx_noname).reset_index(drop = True)

nan_counts = protein_df.isna().sum(axis = 1)
drop_idx_70 = [i for i,j in enumerate(nan_counts) if j >= 0.30*protein_df.shape[1]-1] #drop rows with >30% nans
protein_df = protein_df.drop(drop_idx_70).reset_index(drop = True)

protein_df = protein_df.fillna(value = 0) #impute missing data with 0

protein_df = protein_df.groupby('T: Gene names',as_index = False).median() #replace duplicate entries with the median 

protein_names = protein_df['T: Gene names'].tolist() # get axis names
patient_id = protein_df.columns.tolist()[1:] 

scaler = StandardScaler() #scaling
protein_df_values = protein_df.drop(columns = ['T: Gene names']).values.T
protein_df_values = scaler.fit(protein_df_values).transform(protein_df_values)
protein_df = pd.DataFrame(protein_df_values).T

protein_df.columns = patient_id #reappend axis names
protein_df['Gene_Names'] = protein_names #reappend protein names

TIL_cols = [i for i in protein_df.columns.tolist() if 'TIL' in i]
PD1_cols = [i for i in protein_df.columns.tolist() if 'PD1' in i]

#separate into PD1 and TIL patients
protein_df_PD1 = deepcopy(protein_df[PD1_cols])
protein_df_PD1['Gene_Names'] = protein_names

protein_df_TIL = deepcopy(protein_df[TIL_cols])
protein_df_TIL['Gene_Names'] = protein_names

## Save to CSV

In [54]:
clinical_df.to_csv(module_path + '/data/harel/clinical_df.csv')
protein_df.to_csv(module_path + '/data/harel/protein_df.csv')
protein_df_PD1.to_csv(module_path + '/data/harel/protein_df_PD1.csv')
protein_df_TIL.to_csv(module_path + '/data/harel/protein_df_TIL.csv')
response_df.to_csv(module_path + '/data/harel/response_df.csv')